### RAG기법 활용
### 면접 질문 검색 및 생성 SYSTEM

In [1]:
# Pandas 불러오기
import pandas

In [2]:
# Excel 형식으로 된 DB 불러오기
interview_database = pandas.read_excel("C:/Users/Kamita/anaconda3/SeSAC/mid_project_db_fin.xlsx")

In [3]:
# 질문 column의 결측값 처리 & text형식 file로
interview_database["질문"] = interview_database["질문"].fillna("").astype(str)  # NaN -> 빈 문자열, 숫자 -> 문자열

In [4]:
# 질문을 text형식 file로 변환
with open("interview_questions.txt", "w", encoding="utf-8") as f:
    for question in interview_database["질문"]:
        f.write(question.strip() + "\n")

In [5]:
# TextLoader 활용 text embedding 및 vector화
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# 텍스트 파일 로드
loader = TextLoader("interview_questions.txt", encoding="utf-8")
documents = loader.load()

In [6]:
# 필요한 LangChain 기능 불러오기
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

In [7]:
# 질문 텍스트를 embedding으로 변환
embedding_model = OpenAIEmbeddings()
questions = interview_database["질문"].tolist()
metadata = interview_database[["회사명", "직군"]].to_dict(orient = "records")

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_24668\2524282263.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embedding_model = OpenAIEmbeddings()


In [8]:
# Vector data를 저장할 vector storage 생성
vectorstore = FAISS.from_texts(questions, embedding_model, metadatas = metadata)

KeyboardInterrupt: 

In [ ]:
# 벡터 스토어 저장
vectorstore.save_local("interview_vectorstore")

02. CREATE QUESTIONS

In [9]:
# 벡터 스토어 로드
vectorstore = FAISS.load_local("interview_vectorstore", embedding_model, allow_dangerous_deserialization = True)
retriever = vectorstore.as_retriever()

In [10]:
# LLM 설정
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_24668\869439469.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)


In [11]:
# Retrieval-QA 설정
from langchain.chains import RetrievalQA
retriever_qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever = retriever)

In [12]:
# 질문 검색 함수
def search_questions(company = None, position = None):
    query = f"회사: {company}, 직군: {position}"
    results = retriever.get_relevant_documents(query)   # Document의 객체 list 반환
    return [result.page_content for result in results]  # 각 document의 page content 추출

In [13]:
# 새로운 질문 생성 함수
def generate_new_questions(company, position, related_questions):
    prompt = f"""
    회사: {company}
    직군: {position}
    관련된 기존 질문들: {related_questions}
    위 정보를 바탕으로 새로운 면접 질문을 3개 생성해주세요.
    """
    return llm(prompt)

In [14]:
# 사용자에게 입력받기
company_name = input("회사명을 입력해주세요")
position_name = input("직군/직무를 입력해주세요")
print(f"입력하신 회사는 [{company_name}]이고, 직무는 [{position_name}] 입니다. \n입력하신 정보를 기반으로 질문을 생성합니다")

입력하신 회사는 [삼성전자]이고, 직무는 [AI개발] 입니다. 
입력하신 정보를 기반으로 질문을 생성합니다


In [15]:
# 1. 기존 질문 검색
related_questions = search_questions(company = company_name, position = position_name)
print(f"Related Questions for {company_name} ({position_name}):\n{related_questions}\n")

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_24668\4151313361.py:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  results = retriever.get_relevant_documents(query)   # Document의 객체 list 반환


Related Questions for 삼성전자 (AI개발):
['삼성전자와 같은 디바이스 시스템 개발 회사와 SK C&C같은 SI 회사와 한국투자증권과 같은 자체적 시스템 개발 회사가 있는데, 왜 한국투자증권을 지원하였는가?', '스마트 혁신에 따라 삼성과 애플 그리고 구글 중 스마트폰 시장을 이끌 회사는 어디라 생각하는가?', '경력과 인성 위주 질문 삼성 계열사로 안정성 삼성 계열사', '회사에 대해 궁금한점 평소 인공지능 분야에 관심이 많은지 회사에 지원한 이유']



Failed to multipart ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Monthly unique traces usage limit exceeded"}')trace=2bbdd451-6aaa-4358-94b8-40fbf3761e4f,id=2bbdd451-6aaa-4358-94b8-40fbf3761e4f
Failed to multipart ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Monthly unique traces usage limit exceeded"}')trace=78ec3546-358f-471c-ac89-b1aba950016d,id=78ec3546-358f-471c-ac89-b1aba950016d; trace=2bbdd451-6aaa-4358-94b8-40fbf3761e4f,id=2bbdd451-6aaa-4358-94b8-40fbf3761e4f
Failed to multipart ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/ru

In [18]:
# 2. 새로운 질문 생성
new_questions = generate_new_questions(company = company_name, position = position_name, related_questions = related_questions)
print(f"Generated Questions:\n{new_questions}\n")

TypeError: Got unknown type 
